# Azure AI-Agents mit Unterstützung für das Model Context Protocol (MCP) - Python

Dieses Notebook zeigt, wie Azure AI-Agents mit den Model Context Protocol (MCP)-Tools in Python verwendet werden können. Es wird demonstriert, wie ein intelligenter Agent erstellt werden kann, der externe MCP-Server (wie Microsoft Learn) für erweiterte Funktionen mithilfe einer schlüssellosen Authentifizierung nutzen kann.


## Erforderliche Python-Pakete installieren

Zuerst müssen wir die notwendigen Python-Pakete installieren:
- **azure-ai-projects**: Kern-SDK für Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK zum Erstellen und Verwalten von Agents
- **azure-identity**: Ermöglicht schlüssellose Authentifizierung mit DefaultAzureCredential
- **mcp**: Implementierung des Model Context Protocols für Python


## Vorteile der schlüssellosen Authentifizierung

Dieses Notebook demonstriert die **schlüssellose Authentifizierung**, die mehrere Vorteile bietet:
- ✅ **Keine API-Schlüssel zu verwalten** - Verwendet Azure-Identitätsbasierte Authentifizierung
- ✅ **Erhöhte Sicherheit** - Keine Geheimnisse im Code oder in Konfigurationsdateien gespeichert
- ✅ **Automatische Anmeldeinformationen-Rotation** - Azure übernimmt das Lebenszyklusmanagement der Anmeldeinformationen
- ✅ **Rollenbasierte Zugriffskontrolle** - Nutzt Azure RBAC für fein abgestufte Berechtigungen
- ✅ **Unterstützung für mehrere Umgebungen** - Funktioniert nahtlos in Entwicklungs- und Produktionsumgebungen

Die `DefaultAzureCredential` wählt automatisch die beste verfügbare Anmeldeinformationsquelle aus:
1. **Managed Identity** (bei Ausführung in Azure)
2. **Azure CLI**-Anmeldeinformationen (während der lokalen Entwicklung)
3. **Visual Studio**-Anmeldeinformationen
4. **Umgebungsvariablen** (falls konfiguriert)
5. **Interaktive Browser**-Authentifizierung (als Rückfalloption)


## Einrichtung der schlüssellosen Authentifizierung

**Voraussetzungen für die schlüssellose Authentifizierung:**

### Für lokale Entwicklung:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Für Azure-Umgebungen:
- Aktivieren Sie die **Systemzugewiesene Verwaltete Identität** auf Ihrer Azure-Ressource
- Weisen Sie der verwalteten Identität die entsprechenden **RBAC-Rollen** zu:
  - `Cognitive Services OpenAI User` für den Zugriff auf Azure OpenAI
  - `AI Developer` für den Zugriff auf Azure AI-Projekte

### Umgebungsvariablen (Optional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Keine API-Schlüssel oder Verbindungszeichenfolgen erforderlich!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Erforderliche Bibliotheken importieren

Importieren Sie die notwendigen Python-Module:
- **os, time**: Standard-Python-Bibliotheken für Umgebungsvariablen und Verzögerungen
- **AIProjectClient**: Hauptclient für Azure AI-Projekte
- **DefaultAzureCredential**: Schlüssellose Authentifizierung für Azure-Dienste
- **MCP-bezogene Klassen**: Zum Erstellen und Verwalten von MCP-Tools und Bearbeiten von Genehmigungen


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurieren der MCP-Server-Einstellungen

Richten Sie die MCP-Server-Konfiguration mithilfe von Umgebungsvariablen mit Standardwerten als Fallback ein:
- **MCP_SERVER_URL**: Die URL des MCP-Servers (standardmäßig Microsoft Learn API)
- **MCP_SERVER_LABEL**: Eine Bezeichnung zur Identifizierung des MCP-Servers (standardmäßig "mslearn")

Dieser Ansatz ermöglicht eine flexible Konfiguration in verschiedenen Umgebungen.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Erstellen eines Azure AI Project Clients (Schlüssellose Authentifizierung)

Initialisieren Sie den Azure AI Project Client mit **schlüsselloser Authentifizierung**:
- **endpoint**: Die URL des Azure AI Foundry Projektendpunkts
- **credential**: Verwendet `DefaultAzureCredential()` für eine sichere, schlüssellose Authentifizierung
- **Keine API-Schlüssel erforderlich**: Erkennt und verwendet automatisch die beste verfügbare Anmeldeinformation

**Authentifizierungsablauf:**
1. Überprüft die Managed Identity (in Azure-Umgebungen)
2. Greift auf Azure CLI-Anmeldeinformationen zurück (für lokale Entwicklung)
3. Nutzt bei Bedarf andere verfügbare Anmeldequellen

Dieser Ansatz eliminiert die Notwendigkeit, API-Schlüssel oder Verbindungszeichenfolgen in Ihrem Code zu verwalten.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Erstellen einer MCP-Tool-Definition

Erstellen Sie ein MCP-Tool, das eine Verbindung zum Microsoft Learn MCP-Server herstellt:
- **server_label**: Bezeichner für den MCP-Server
- **server_url**: URL-Endpunkt des MCP-Servers
- **allowed_tools**: Optionale Liste, um einzuschränken, welche Tools verwendet werden können (leere Liste erlaubt alle Tools)

Dieses Tool ermöglicht es dem Agenten, auf die Microsoft Learn-Dokumentation und -Ressourcen zuzugreifen.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Erstellen eines Agenten und Ausführen einer Unterhaltung (Keyless Workflow)

Dieser umfassende Abschnitt zeigt den vollständigen **Keyless-Agent-Workflow**:

1. **KI-Agent erstellen**: Einen Agenten mit dem GPT-4.1 Nano-Modell und MCP-Tools einrichten
2. **Thread erstellen**: Einen Gesprächs-Thread für die Kommunikation einrichten
3. **Nachricht senden**: Den Agenten nach den Unterschieden zwischen Azure OpenAI und OpenAI fragen
4. **Toolgenehmigungen verwalten**: MCP-Toolaufrufe bei Bedarf automatisch genehmigen
5. **Ausführung überwachen**: Den Fortschritt des Agenten verfolgen und erforderliche Aktionen durchführen
6. **Ergebnisse anzeigen**: Das Gespräch und die Details zur Toolnutzung anzeigen

**Keyless-Funktionen:**
- ✅ **Keine fest codierten Geheimnisse** - Alle Authentifizierungen werden über Azure Identity abgewickelt
- ✅ **Standardmäßig sicher** - Verwendet rollenbasierte Zugriffskontrolle
- ✅ **Vereinfachte Bereitstellung** - Keine Verwaltung von Zugangsdaten erforderlich
- ✅ **Audit-freundlich** - Alle Zugriffe werden über Azure Identity nachverfolgt

Der Agent wird MCP-Tools verwenden, um auf Microsoft Learn-Ressourcen zuzugreifen, mit voller Sicherheit und ohne API-Schlüsselverwaltung.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
